In [1]:
!pip install gensim

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 24.2 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 101.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 104.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 68.0 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 93.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 91.2 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 97.4 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=102298107814ceb0ab39e402aec025e676846ebb1170f3ccb06e2eedbab45c24
  Stored in directory: /home/tione/.cache/pip/wheels/54/7f/ad/18624c030ede84ebb5f4bcd3a6cdfab71662eccc4f4042df14
Successfully built smart-open
  Attempting unin

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gc

In [3]:
#读数据
# train_ad = pd.read_csv('data/train_preliminary/ad.csv')
train_click_log = pd.read_csv('data/train_preliminary/click_log.csv')
# train_user = pd.read_csv('data/train_preliminary/user.csv')

# test_ad = pd.read_csv('data/test/ad.csv')
test_click_log = pd.read_csv('data/test/click_log.csv')

In [4]:
# train_test_ad = train_ad.append(test_ad)
train_test_click_log = train_click_log.append(test_click_log)
# train_test_data = pd.merge(train_test_click_log,train_test_ad,how='outer')
# del train_ad, test_ad, train_click_log, test_click_log
# gc.collect()

In [5]:
train_test_click_log = train_test_click_log.sort_values(by=['user_id', 'time'])
train_test_click_log

,time,user_id,creative_id,click_times
26196513,20,1,821396,1
26196514,20,1,209778,1
26196515,20,1,877468,1
26196517,39,1,1683713,1
26196524,40,1,122032,1
...,...,...,...,...
14648696,75,4000000,3596158,1
14648699,75,4000000,3642395,1
14648674,76,4000000,366858,1
14648681,76,4000000,3333680,1


In [6]:
train_test_id = train_test_click_log[['user_id']].drop_duplicates()
train_test_id

,user_id
26196513,1
12979024,2
29916605,3
1685803,4
18556101,5
...,...
31372714,3999996
14648631,3999997
14648665,3999998
31372771,3999999


In [7]:
train_test_features = train_test_click_log[['user_id', 'creative_id']]
train_test_features

,user_id,creative_id
26196513,1,821396
26196514,1,209778
26196515,1,877468
26196517,1,1683713
26196524,1,122032
...,...,...
14648696,4000000,3596158
14648699,4000000,3642395
14648674,4000000,366858
14648681,4000000,3333680


In [8]:
#造句
def get_sentences(df_, key, name):
    sentences = df_.groupby([key]).apply(lambda x: x[name].tolist()).tolist()
    sentences3 = []
    for i in range(len(sentences)):
        sentences2 = list(map(str, sentences[i]))
        sentences3.append(sentences2)
    return sentences3

sentences = get_sentences(train_test_features, 'user_id', 'creative_id')
len(sentences)

1900000

In [9]:
#w2c
def get_embedding_df(df_, sentences, flag, L, name):
    if flag == True:
        model = Word2Vec(sentences=sentences, size=L, window=10, min_count=2, workers=10, iter=10)
        model.save('output/model/' + name + '_w2c.model')
        print("模型保存完成！！！")
    model = Word2Vec.load('output/model/' + name + '_w2c.model')
    values=set(df_[name].values)
    w2v=[]

    for v in values:
        try:
            a=[int(v)]
            a.extend(model[str(v)])
            w2v.append(a)
        except:
            pass

    out_df=pd.DataFrame(w2v)
    del w2v
    gc.collect()
    out_df.columns = [name]+[name+'_embe_{}'.format(i) for i in range(L)]
#     embedding = pd.merge(df_,out_df,on=name,how='left')
#     del out_df
#     gc.collect()
    return out_df


In [10]:
out_df = get_embedding_df(train_test_features, sentences, False, 32, 'creative_id')
out_df

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


,creative_id,creative_id_embe_0,creative_id_embe_1,creative_id_embe_2,creative_id_embe_3,creative_id_embe_4,creative_id_embe_5,creative_id_embe_6,creative_id_embe_7,creative_id_embe_8,...,creative_id_embe_22,creative_id_embe_23,creative_id_embe_24,creative_id_embe_25,creative_id_embe_26,creative_id_embe_27,creative_id_embe_28,creative_id_embe_29,creative_id_embe_30,creative_id_embe_31
0,1,-0.537792,-0.343738,-0.169357,0.170321,-0.142634,-0.133415,0.015727,0.575709,-0.714908,...,-0.395647,0.067353,0.055399,0.074493,-0.036078,0.190803,0.714070,-0.100935,0.309329,0.063770
1,4,-0.525975,-0.368707,-0.289037,-0.434968,0.039553,-0.355471,-0.223031,0.464887,-0.182532,...,-0.835788,-0.075186,0.597538,0.056204,0.172193,0.052432,0.639376,-0.063182,0.817930,-0.240091
2,7,-0.077517,-0.140333,-0.011366,-0.080992,-0.283651,-0.032912,-0.273827,0.224146,-0.195711,...,-0.008073,0.029026,-0.045505,0.152764,-0.091590,-0.029852,0.127660,0.134912,0.115477,0.092430
3,10,-0.112068,-0.231497,-0.043593,-0.166035,-0.165059,-0.094334,-0.007987,0.236783,-0.223286,...,-0.071596,-0.043822,0.119022,0.052398,0.081226,0.157128,0.278553,-0.083885,0.322027,0.105955
4,13,-0.245735,-0.481754,0.052759,-0.122637,-0.141151,-0.118338,-0.158126,0.259479,-0.034940,...,-0.441856,-0.191594,0.339228,-0.044013,0.141239,0.283528,0.169891,-0.121109,0.240923,-0.135798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086581,4445710,-0.173163,-0.087752,-0.091049,-0.142106,-0.060819,-0.118301,0.030556,0.099103,-0.108970,...,-0.163997,0.010826,0.143995,-0.022708,-0.038149,0.012862,0.114340,-0.087390,0.194263,0.004597
2086582,4445711,-0.240102,-0.245562,-0.012533,-0.109751,-0.185693,-0.186433,0.019425,0.077360,-0.092199,...,-0.265054,-0.122758,0.133444,0.038254,0.119017,0.043823,0.130895,0.005913,0.025594,-0.061541
2086583,4445715,-0.101917,-0.142415,-0.086243,-0.042997,-0.112699,-0.129123,-0.068477,0.088054,0.048495,...,-0.095513,-0.085330,0.184931,0.029114,-0.074619,0.087402,0.137976,-0.162941,0.240736,0.019241
2086584,4445718,-0.191890,-0.157027,-0.046568,-0.051968,-0.124664,-0.124049,-0.030051,0.041827,-0.049231,...,-0.142483,-0.072065,0.168712,-0.041088,-0.097093,0.046425,0.174572,-0.073851,0.151137,0.054175


In [13]:
out_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086586 entries, 0 to 2086585
Data columns (total 33 columns):
 #   Column               Dtype  
---  ------               -----  
 0   creative_id          int64  
 1   creative_id_embe_0   float64
 2   creative_id_embe_1   float64
 3   creative_id_embe_2   float64
 4   creative_id_embe_3   float64
 5   creative_id_embe_4   float64
 6   creative_id_embe_5   float64
 7   creative_id_embe_6   float64
 8   creative_id_embe_7   float64
 9   creative_id_embe_8   float64
 10  creative_id_embe_9   float64
 11  creative_id_embe_10  float64
 12  creative_id_embe_11  float64
 13  creative_id_embe_12  float64
 14  creative_id_embe_13  float64
 15  creative_id_embe_14  float64
 16  creative_id_embe_15  float64
 17  creative_id_embe_16  float64
 18  creative_id_embe_17  float64
 19  creative_id_embe_18  float64
 20  creative_id_embe_19  float64
 21  creative_id_embe_20  float64
 22  creative_id_embe_21  float64
 23  creative_id_embe_22  float64
 24

In [11]:
out_df.to_csv('output/model/creative_id_embedding.csv', index=False)